In [1]:
import pandas as pd
df = pd.read_csv("cases.csv")
df

,Case_ID,no_of_hearings,no_of_advocates,no_of_issues,no_of_laws,no_of_precedents,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,C001,3,3,2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C002,4,3,3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C003,2,3,3,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C004,4,3,3,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C005,5,6,2,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,C096,3,6,2,20,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,C097,4,6,2,11,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,C098,20,5,3,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,C099,3,3,2,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
X = df[["no_of_advocates","no_of_issues","no_of_laws","no_of_precedents"]]

In [3]:
y = df["no_of_hearings"]

In [4]:
X

,no_of_advocates,no_of_issues,no_of_laws,no_of_precedents
0,3,2,2,0
1,3,3,3,0
2,3,3,1,0
3,3,3,2,0
4,6,2,1,0
...,...,...,...,...
95,6,2,20,0
96,6,2,11,0
97,5,3,9,0
98,3,2,8,0


In [5]:
y

0      3
1      4
2      2
3      4
4      5
      ..
95     3
96     4
97    20
98     3
99     3
Name: no_of_hearings, Length: 100, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.1, random_state = 123)

In [8]:
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [9]:
gbm = xgb.XGBRegressor()
reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]
                            ,'max_depth': [3,4,6], 'n_estimators': [500,1000]}, verbose=1)
reg_cv.fit(train_X,train_y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                     

In [10]:
reg_cv.best_params_

{'colsample_bytree': 1.0,
 'max_depth': 4,
 'min_child_weight': 1.0,
 'n_estimators': 500}

In [11]:
gbm = xgb.XGBRegressor(**reg_cv.best_params_)
gbm.fit(train_X,train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=1.0, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [12]:
predictions = gbm.predict(test_X)
predictions

array([ 3.2332668,  2.6238482, 15.747566 ,  4.520479 ,  7.983972 ,
       10.543372 , 25.563702 , 19.460897 ,  3.0026367,  5.8769093],
      dtype=float32)

In [13]:
gbm.score(test_X,test_y)

0.1451962221954637

In [14]:
gbm.score(train_X,train_y)

0.9978153440058616

In [15]:
import numpy as np

In [16]:
from sklearn.metrics import r2_score
r2 = np.sqrt(r2_score(test_y, predictions))
print("r2 : % f" %(r2))

r2 :  0.381046


In [17]:
new_data = [40,10,1,7]

In [18]:
new_data_arr = np.asarray([new_data])

In [19]:
yhat = gbm.predict(new_data_arr)
yhat

array([21.913065], dtype=float32)

In [20]:
new_data = [40,10,1,7]

In [21]:
new_data_arr = np.asarray([new_data])

In [22]:
yhat = gbm.predict(new_data_arr)
yhat

array([21.913065], dtype=float32)

In [23]:
new_data = [14,3,27,0]

In [24]:
new_data_arr = np.asarray([new_data])

In [25]:
yhat = gbm.predict(new_data_arr)
yhat

array([5.000422], dtype=float32)

In [26]:
new_data = [2,3,3,1]

In [27]:
new_data_arr = np.asarray([new_data])

In [28]:
yhat = gbm.predict(new_data_arr)
yhat

array([8.999895], dtype=float32)

In [29]:
df["case_complexity"] = df["no_of_issues"]+df["no_of_laws"]+df["no_of_precedents"]

In [30]:
mean = df["case_complexity"].mean()

In [31]:
df["complexity_by_mean"] = df["case_complexity"]/mean

In [38]:
df["case_complexity"].describe()

count    100.000000
mean      12.430000
std       11.666455
min        1.000000
25%        5.000000
50%        8.000000
75%       15.000000
max       73.000000
Name: case_complexity, dtype: float64

In [44]:
df["normalised"] = ((df["case_complexity"]-1.000000)*6)/(73.000000-1)

In [45]:
df["normalised"].describe()

count    100.000000
mean       0.952500
std        0.972205
min        0.000000
25%        0.333333
50%        0.583333
75%        1.166667
max        6.000000
Name: normalised, dtype: float64

In [46]:
df["normalised"]

0     0.250000
1     0.416667
2     0.250000
3     0.333333
4     0.166667
        ...   
95    1.750000
96    1.000000
97    0.916667
98    0.750000
99    0.583333
Name: normalised, Length: 100, dtype: float64

In [32]:
import pickle
pickle_out= open("cases.pkl","wb")
pickle.dump(gbm,pickle_out)
pickle_out.close()

In [33]:
import pickle 
filename = 'cases.sav'
pickle.dump(gbm,open(filename , 'wb'))

In [34]:
loaded_model = pickle.load(open('cases.sav','rb'))

In [35]:
input_data = (40,10,1,7)
inp_as_nparr = np.asarray(input_data)
input_data_reshaped = inp_as_nparr.reshape(1,-1)
prediction = loaded_model.predict(input_data_reshaped)
complexity = (input_data[1]+input_data[2]+input_data[3])/12
print(complexity)
print(prediction)

1.5
[21.913065]


In [36]:
X.columns

Index(['no_of_advocates', 'no_of_issues', 'no_of_laws', 'no_of_precedents'], dtype='object')